# FRE6711 Quantitative Portfolio Management Memo for the Final Project

This project aims 1) to build a factor-based model allocation namely a Long/Short Global Macro Strategy with a Target Beta and 2) to evaluate its sensitivity to variations of Beta and its sensitivity to the length of the estimators for covariance matrix and the expected returns under different market scenarios.

#### Please run the code of Part 1 - Part 5.1 directly. Then, set the target backtesting periods in the 1st cell of Part 5.2, and run all of the rest of code.

In [ ]:
%%capture
import pandas as pd
import numpy as np
import copy
import itertools
import statsmodels.api as sm # for linear regresion
from cvxopt import spmatrix
from cvxopt import matrix
from cvxopt.solvers import qp
from scipy.stats import skew, kurtosis
from scipy.stats import norm
import seaborn as sns
sns.set() 
%matplotlib inline     
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates

import warnings
warnings.filterwarnings("ignore")

In [ ]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (24, 12),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
plt.rcParams.update(params) 

## Part 1: Import & Process Data 

## Part 1-1 Import Data
- We fetch data from the websites and save it as csv files:

In [ ]:
url = 'https://drive.google.com/uc?export=download&id=1zYT4qlPzdNb4c4b4cFBaFzHxZCXdIIGt'
adjclose = pd.read_csv(url, parse_dates=['Date'], index_col = 'Date', dayfirst = True)
logReturn = np.log(adjclose).diff().dropna()
logReturn

In [ ]:
url = 'https://drive.google.com/uc?export=download&id=13cB_WCvBqF3b9r8fXl3ar0LEHwDW5QeR'
factors = pd.read_csv(url, index_col = 0)
factors.index = pd.to_datetime(factors.index, format = '%Y%m%d')
factors = factors.iloc[1:-1,:]
factors

## Part 2: Build French Fama 3-factor model

In [ ]:
def FF3_Reg(logReturn, factors):
    
    excess_logReturn = logReturn.values - factors['RF'].values.reshape(len(factors['RF']),1)
    
    reg = pd.DataFrame(index = logReturn.index, columns = logReturn.columns)
    for i in range(len(logReturn.columns)):
        x = factors[["Mkt-RF", "SMB", "HML"]]
        y = excess_logReturn[:,i]
        x = sm.add_constant(x)
        model = sm.OLS(y, x)
        olsres = model.fit()
        reg.iloc[:,i] = olsres.predict(x) + factors['RF']
    
    return reg

## Part 3: Optimization

In [ ]:
def Optimization(logReturn, factors, w_previous, today, days_return, days_cov):
    n = len(logReturn.columns)
    today_idx = logReturn.index.get_indexer([today])[0]
    df_expRet = FF3_Reg(logReturn.iloc[today_idx-days_cov:today_idx, :], factors.iloc[today_idx-days_cov:today_idx, :])
    P = matrix(np.cov(df_expRet.T))
    df_expRet = FF3_Reg(logReturn.iloc[today_idx-days_return:today_idx, :], factors.iloc[today_idx-days_return:today_idx, :])
    q = matrix(-np.sum(df_expRet, axis = 0)- w_previous@P) # considering the weights of the last period

    G1 = matrix(0.0, (n,n))
    G1[::n+1] = 1.0
    G = matrix([G1, -G1])
    h = matrix(2.0, (n*2,1))

    beta_matrix = []
    for i in range(len(logReturn.columns)):
        beta_matrix.append(np.cov(df_expRet.iloc[:,i], logReturn['SPY'][today_idx-days_return:today_idx])[1,0])

    beta_matrix = matrix(np.array(beta_matrix)/ np.var(logReturn['SPY'][today_idx-days_return:today_idx], ddof=1))
    A1 = matrix(1.0, (1,n))
    A = matrix([beta_matrix.T, A1])
    b = matrix([Beta_target, 1.0])
    w = qp(P, q, G, h, A, b)['x']
    return w

## Part 4: Output Weight Data

In [ ]:
# days_return period could be 20, 80, 140
days_return_L = [20, 80, 140]
days_cov_L = [20, 80, 140]

# Beta_target could be -1, -0.5, 0, 0.5, 1, 1.5
Beta_target_L = [-1, 0, 1]

weight_df = pd.DataFrame(columns = ['return days', 'cov days', 'Beta', 'Start Date'] + logReturn.columns.tolist())

for days_return, days_cov, Beta_target in itertools.product(days_return_L, days_cov_L, Beta_target_L):
    print('return days: ', days_return, 'cov days: ', days_cov, 'Beta: ', Beta_target)
    day_pass = days_return if days_return > days_cov else days_cov
    dates = pd.DataFrame(logReturn.index)
    dates = dates.iloc[day_pass+1:,:]
    g = dates.groupby(pd.Grouper(key='Date', freq='W'))  # frequency = weekly
    dates = [group.iloc[0,0] for _,group in g]

    n = len(logReturn.columns)
    w_previous = np.ones(n)/n
    for today in dates: 
        w = Optimization(logReturn, factors, w_previous, today, days_return, days_cov)
        weight_df = weight_df.append({'return days':days_return, 'cov days': days_cov, 'Beta': Beta_target, 'Start Date': today}, ignore_index=True)
        weight_df.iloc[-1,4:] = pd.Series(w).values
        w_previous = weight_df.iloc[-1,4:].values

In [ ]:
df_benchmark = pd.DataFrame(columns = ['return days', 'cov days', 'Beta', 'Start Date']+ logReturn.columns.tolist())
df_benchmark['Start Date'] = logReturn.index
df_benchmark.loc[:,['return days', 'cov days', 'Beta']] = 'SPY'
df_benchmark['SPY'] = 1
df_benchmark.fillna(0, inplace = True)
weight_df = pd.concat([weight_df, df_benchmark], axis = 0)
weight_df = weight_df.set_index(['return days' , 'cov days', 'Beta'])

In [ ]:
weight_df

## Part 5-1: Backtesting Funtions

In [ ]:
def BacktestingDaily(df_returns, df_weights): 
    df_weights = df_weights.set_index(df_weights.iloc[:,0])
    df_weights.drop(columns = df_weights.columns[0], axis=1, inplace=True)
    
    daily_weights = pd.DataFrame(index = df_returns.index)
    daily_weights = pd.concat([daily_weights, df_weights], axis= 1)
    daily_weights.fillna(method="ffill", inplace = True) # fill the weights to the following cells with no datum
    returns = np.sum(df_returns * daily_weights, axis = 1)
    PV = (returns+1).cumprod() 
    
    daily_report = pd.DataFrame({'PV': PV, 'Daily Returns': returns})
    daily_report.iloc[0,1] =  daily_report.iloc[0,0] - 1 
    return daily_report

In [ ]:
def cvar_parametric(portofolioReturns, portfolioStd, alpha=5, dof=6):
    # Reference Website: Jonathon Emerick, QuantPy website
    # https://quantpy.com.au/risk-management/value-at-risk-var-and-conditional-var-cvar/
    CVaR = portofolioReturns - norm.pdf(norm.ppf(alpha/100)) * portfolioStd / (alpha/100) 
    return CVaR
        
def modified_VaR(returns_mean, sigma, skew, kurt, alpha=5):
    z_score = norm.ppf(alpha/100)
    Z = z_score + (z_score ** 2 - 1) * skew / 6 + (z_score ** 3 - 3 * z_score) * kurt / 24 - (2 * z_score ** 3 - 5 * z_score) * skew ** 2 / 36
    return returns_mean + (sigma * Z)

## Part 5-2: Backtesting 
Please set the period in the following 1st section.

In [ ]:
# Split into three periods
#    - Before Crisis (bc): 2007/1/1 - 2008/1/1
#    - During Crisis (dc): 2008/1/1 - 2011/1/1
#    - After Crisis (ac): 2011/1/1 - 2022/9/30
period_start = "2008/1/1"
period_end = "2011/1/1"

In [ ]:
weights = weight_df.loc[(weight_df['Start Date'] >= period_start) & (weight_df['Start Date'] <= period_end),:]
LogReturn = logReturn.loc[(logReturn.index >= period_start) & (logReturn.index <= period_end),:]
g = weights.groupby(level=[0,1,2])  
dfList = [group for _,group in g]

c = ['Cumulated PnL', 'Daily Arithmetic Mean Return', 'Daily Geometric Mean Return', 'Daily Min Return',
        'Max 10 days Drawdown', 'Volatility', 'Sharpe Ratio', 'Skewness', 'Kurtosis', 'Modified VaR', 'CVaR']
report = pd.DataFrame(index = weights.index.unique(), columns = c) # Create Summary Report
PVPlot = pd.DataFrame(index = weights.index.unique()).T # DataFrame for ploting the cummulated daily PnL
dailyReturnPlot = pd.DataFrame(index = weights.index.unique()).T  # DataFrame for ploting the distribution of daily return
for i in range(len(dfList)):
        w = dfList[i]
        r = LogReturn
        DailyReport= BacktestingDaily(r, dfList[i])
        rep = AnnulizedAnalysis(DailyReport)
        report.iloc[i,:] = rep.iloc[0,:]
        PVPlot.iloc[:,i] = DailyReport['PV'] * 100
        dailyReturnPlot.iloc[:,i] = DailyReport['Daily Returns'] * 100

### Summary Report

In [ ]:
report

### Cummulated Daily PnL Graph

In [ ]:
add_SPY = False

In [ ]:
if add_SPY == True:
    PVPlot.iloc[:,:].plot(grid = True, ylabel = "Cumulative PnL")
else:
    PVPlot.iloc[:,:-1].plot(grid = True, ylabel = "Cumulative PnL")

plt.legend(loc = "best")
plt.legend(bbox_to_anchor = (0,0,1.15,1))

In [ ]:
PVPlot_SPY = PVPlot.loc[:,'SPY']
PVPlot2 = PVPlot.drop(columns=['SPY'])
col_List = [('return days','cov days', 'Beta'), ('return days', 'Beta','cov days'),('cov days', 'Beta','return days')]
for _, element in enumerate(col_List):
    PVPlot_temp2 = copy.deepcopy(PVPlot2)
    PVPlot_temp2 = PVPlot_temp2.T
    PVPlot_temp2.reset_index(inplace = True)
    PVPlot_temp2 = pd.melt(PVPlot_temp2,id_vars=['return days','cov days', 'Beta'])
    
    if add_SPY == True:
        PVPlot_temp = pd.DataFrame(columns=PVPlot2.columns).T.reset_index()
        PVPlot_temp[element[2]] = 'SPY'
        PVPlot_temp.drop(PVPlot_temp.tail(1).index,inplace=True)
        PVPlot_temp = PVPlot_temp.set_index(['return days' , 'cov days', 'Beta'])
        PVPlot_temp = pd.DataFrame(index = PVPlot2.index, columns = PVPlot_temp.index.unique())
        PVPlot_temp.iloc[:,:] = PVPlot_SPY
        PVPlot_temp = PVPlot_temp.T
        PVPlot_temp.reset_index(inplace = True)
        PVPlot_temp = pd.melt(PVPlot_temp,id_vars=['return days','cov days', 'Beta'])
    
        PVPlot_temp = pd.concat([PVPlot_temp, PVPlot_temp2], axis = 0).reset_index(drop=True)
        pd.to_datetime(PVPlot_temp['Date'], format = '%Y-%m-%d')
        distribution = sns.FacetGrid(PVPlot_temp, col =element[0] , row = element[1], hue = element[2], height = 5)
    else:
        distribution = sns.FacetGrid(PVPlot_temp2, col =element[0] , row = element[1], hue = element[2], height = 5)
    distribution.map_dataframe(sns.lineplot,'Date', 'value', alpha = .7)
    distribution.add_legend()
    distribution.set_axis_labels('Date', 'value')
    distribution.axes[0,0].xaxis.set_major_locator(mdates.YearLocator(base = 5))

### Daily Return Distribution Graph

In [ ]:
add_SPY = False

In [ ]:
dailyReturnPlot1 = copy.deepcopy(dailyReturnPlot)
dailyReturnPlot1.columns = dailyReturnPlot1.columns.to_flat_index()
if add_SPY == True:
    ax = sns.kdeplot(data=dailyReturnPlot1.iloc[:,:])
else:
    ax = sns.kdeplot(data=dailyReturnPlot1.iloc[:,:-1])
sns.move_legend(ax, loc = 'best', bbox_to_anchor = (0,0,1.15,1))

In [ ]:
dailyReturnPlot_SPY = dailyReturnPlot.loc[:,'SPY']
dailyReturnPlot2 = dailyReturnPlot.drop(columns=['SPY'])
col_List = [('return days','cov days', 'Beta'), ('return days', 'Beta','cov days'),('cov days', 'Beta','return days')]
for _, element in enumerate(col_List):
    dailyReturnPlot_temp2 = copy.deepcopy(dailyReturnPlot2)
    dailyReturnPlot_temp2 = dailyReturnPlot_temp2.T
    dailyReturnPlot_temp2.reset_index(inplace = True)
    dailyReturnPlot_temp2 = pd.melt(dailyReturnPlot_temp2,id_vars=['return days','cov days', 'Beta'])  
    if add_SPY == True:
        dailyReturnPlot_temp = pd.DataFrame(columns=dailyReturnPlot2.columns).T.reset_index()
        dailyReturnPlot_temp[element[2]] = 'SPY'
        dailyReturnPlot_temp.drop(dailyReturnPlot_temp.tail(1).index,inplace=True)
        dailyReturnPlot_temp = dailyReturnPlot_temp.set_index(['return days' , 'cov days', 'Beta'])
        dailyReturnPlot_temp = pd.DataFrame(index = dailyReturnPlot2.index, columns = dailyReturnPlot_temp.index.unique())
        dailyReturnPlot_temp.iloc[:,:] = dailyReturnPlot_SPY
        dailyReturnPlot_temp = dailyReturnPlot_temp.T
        dailyReturnPlot_temp.reset_index(inplace = True)
        dailyReturnPlot_temp = pd.melt(dailyReturnPlot_temp,id_vars=['return days','cov days', 'Beta'])

        dailyReturnPlot_temp = pd.concat([dailyReturnPlot_temp, dailyReturnPlot_temp2], axis = 0).reset_index(drop=True)
        pd.to_datetime(dailyReturnPlot_temp['Date'], format = '%Y-%m-%d') 
    
        distribution = sns.FacetGrid(dailyReturnPlot_temp, col =element[0] , row = element[1], hue = element[2], height = 4)
    else:
        distribution = sns.FacetGrid(dailyReturnPlot_temp2, col =element[0] , row = element[1], hue = element[2], height = 4)
    
    distribution.map(sns.kdeplot,'value', alpha = .7)
    distribution.add_legend()
    distribution.set_axis_labels('value', 'count')